In [1]:
import sys
sys.path.append('..')
from tool import *

In [ ]:
def read_file(file_name,
              ocean_mask=True,
              snow_mask=False):
    hdf_obj = SD(file_name)
    WSA=hdf_obj.select('WSA_shortwave')[:].astype(np.float32)
    BSA=hdf_obj.select('BSA_shortwave')[:].astype(np.float32)
    qa=hdf_obj.select('QC')[:]
    extracted_bits = (qa & 0x1C) >> 2
    output_image = extracted_bits.astype(np.uint8)
    if ocean_mask:
        ocean=((output_image==2)|(output_image==3)|(output_image==4))
        WSA[ocean==1]=np.nan
        BSA[ocean==1]=np.nan
    if snow_mask:
        snow=((output_image==1)|(output_image==5))
        WSA[snow==1]=np.nan
        BSA[snow==1]=np.nan
    return WSA,BSA

def resample_wb_fraction(img,width=7200,height=3600,resample_alg = gdalconst.GRIORA_Bilinear):
    driver = gdal.GetDriverByName('MEM')
    src_ds = driver.Create('', img.shape[1],img.shape[0], 1, 6)
    src_ds.GetRasterBand(1).WriteArray(img)
    src_ds.GetRasterBand(1).SetNoDataValue(2)
    src_ds.FlushCache()
    data = src_ds.GetRasterBand(1).ReadAsArray(buf_xsize=width,buf_ysize=height,resample_alg = resample_alg)
    return data

def get_h0_a(lon, lat, date, time, timezone):
    date = datetime.datetime.strptime(date, '%Y/%m/%d')
    Dn = int(date.strftime('%j'))
    time = datetime.datetime.strptime(time, '%H:%M')
    Hour = time.hour + time.minute/60.0
    gamma = 2*pi*(Dn - 1 + (Hour - 12)/24)/365
    eqtime = 229.18*(0.000075 + 0.001868*cos(gamma) - 0.032077*sin(gamma) - 0.014615*cos(2*gamma) - 0.040849*sin(2*gamma))
    f1 = 0.006918
    f2 = 0.399912*cos(gamma)
    f3 = 0.070257*sin(gamma)
    f4 = 0.006758*cos(gamma*2)
    f5 = 0.000907*sin(gamma*2)
    f6 = 0.002697*cos(gamma*3)
    f7 = 0.001480*sin(gamma*3)
    decl = f1 - f2 + f3 - f4 + f5 - f6 + f7
    time_offset = eqtime + 4*lon - 60*timezone
    tst = Hour*60 + time_offset
    ha = (tst/4 - 180)
    snoon = (720 - 4*lon - eqtime)/60 + timezone
    lat = radians(lat)
    ha = radians(ha) 
    h0 = asin(sin(lat)*sin(decl) + cos(lat)*cos(decl)*cos(ha))
    sza=90-degrees(h0)
    if sza>85:
        sza_L=0
    elif sza>70:
        sza_L=1
    else:
        sza_L=2
    return sza,sza_L

In [ ]:
#Calculate annual land surface blue-sky albedo based on GLASS product
fs=glob(f'/mnt/hzy_ssd2/GLASS/GLASS02B03.V50.A2001*.hdf')
fs.sort()
doys=[]
for i in range(len(fs)):
    doys.append(fs[i].split('.')[2][-3:])
doys_t=[datetime.datetime.strptime('2001/01/01', '%Y/%m/%d')+datetime.timedelta(days=int(i)-1) for i in doys]
def get_md(date_time):
    kk=date_time.strftime("%Y-%m-%d").split('-')
    return [int(kk[1]),int(kk[2])]
doys_md=[get_md(i) for i in doys_t]

sza_ar=np.zeros((46,3600,7200))
for l in [i*0.05 for i in range(1800,-1800,-1)]:
    for z in range(len(doys_md)):
        md=doys_md[z]
        sza_info=get_h0_a(0, l, f'{y}/{md[0]}/{md[1]}', '12:00', 0.0)
        sza_ar[z,1800-round(l/0.05),:]=sza_info[0]
sza_L=np.zeros((46,3600,7200))
sza_L[sza_ar>70]=1
sza_L[sza_ar>85]=2 

WB_sky_fraction=torch.load("/mnt/hzy_ssd2/white_sky_fraction/white_sky_fraction2.pth")
for y in range(2001,2021):
    fs=glob(f'/mnt/hzy_ssd2/GLASS/GLASS02B03.V50.A{y}*.hdf')
    fs.sort()
    blueSAs=[]
    for i in tqdm(range(len(fs))):
        index=(y-2001)*12+doys_md[i][0]-1
        fraction=resample_wb_fraction(WB_sky_fraction[index])
        sza_ld=sza_L[i]
        file_name=fs[i]
        WSA,BSA=read_file(file_name)
        blueSA=WSA*fraction+BSA*(1-fraction)
        blueSA[sza_ld>0]=np.nan
        blueSAs.append(blueSA)
    blueSAs=np.stack(blueSAs)
    blueSAs_y=np.nanmean(blueSAs,axis=0)
    torch.save(blueSAs_y,f'/mnt/hzy_ssd2/GLASS_result/blueSAs_{y}_gt70_withuncertaintyandprior.pth')

In [ ]:
#Calculate annual land surface mean blue-sky albedo based on GLASS product
R = 6371007.181	
pixel_with=0.05
lats=[90-i*0.05 for i in range(3600)]
pixels_area=[(pi/180.0)*R*R*abs(math.sin((lat)/180.0*pi) - math.sin((lat-pixel_with)/180.0*pi)) * pixel_with for lat in lats]
area_map=np.array([[1]*7200])*(np.array([pixels_area]).T)/1000000

kk_blueSA=[]
for y in tqdm(range(2001,2021)):
    area_map_temp=area_map.copy()
    blueSA_y=torch.load(f'/mnt/hzy_ssd2/GLASS_result/blueSAs_{y}_gt70_withuncertaintyandprior.pth')
    area_map_temp[np.isnan(blueSA_y)]=np.nan
    kk_blueSA.append(np.nansum(area_map_temp*blueSA_y)/np.nansum(area_map_temp))
kk_blueSA_season=np.zeros((4,20))
for y in tqdm(range(2001,2021)):
    blueSA_y=torch.load(f'/mnt/hzy_ssd2/GLASS_result/blueSAs_{y}_gt70_withuncertaintyandprior_season.pth')
    season=[]
    for i in range(4):
        area_map_temp=area_map.copy()                                                               
        area_map_temp[np.isnan(blueSA_y[i])]=np.nan
        kk_blueSA_season[i,y-2001]=np.nansum(area_map_temp*blueSA_y[i])/np.nansum(area_map_temp)
torch.save([kk_blueSA,kk_blueSA_season],'../data/GLASS_curve.pth')

In [ ]:
#Export plot figure
area_all=[]
for landcover_type in ['LCCS1','LCCS2','LCCS3','IGBP']:
    if landcover_type=='LCCS2':
        lc_len=10
    elif landcover_type=='LCCS3':
        lc_len=9
    elif landcover_type=='IGBP':
        lc_len=16
    elif landcover_type=='LCCS1':
        lc_len=15 
    stat=torch.load(f'/data2/hzy/albedo2/data/stat_sw-mean_sza-70_band-shortwave_{landcover_type}.pth')
    _,sza_L=torch.load(root_path+'sza_and_szaL.pth')
    area_all_sub=np.zeros((12,20,lc_len+1,180,360)).astype(np.float32)
    mask=np.zeros((12,20,lc_len+1,180,360))
    for y in range(2001,2021):
        for m in range(1,13):
            area_all_sub[m-1,y-2001]=stat[f'{y}-{m}-typearea']
            mask[m-1,y-2001]=np.expand_dims(sza_L[y-2001,m-1],0).repeat(lc_len+1,axis=0)
    area_all_sub=month_fill(area_all_sub,mask>=1)
    area_all.append(area_all_sub[:,:,-1])
area_all=np.mean(np.stack(area_all),axis=0)
data=torch.load('../data/curve.pth')
plt.figure(figsize=(20,11),dpi=600) 
kk=[]
std=[]
mins=[]
maxs=[]
for i in ['LCCS1','LCCS2','LCCS3','IGBP']:
    kk.append(data[f'G_snow_{i}'])
    std.append(data[f'G_snow_{i}_std'])
    mins.append(data[f'G_snow_{i}_min'])
    maxs.append(data[f'G_snow_{i}_max'])
kk=np.mean(np.stack(kk),axis=0)
maxs=np.max(np.stack(maxs),axis=0)
mins=np.max(np.stack(mins),axis=0)
stds=np.mean(np.stack(std),axis=0)
kk2=[0]+[kk[j]-kk[0] for j in range(1,len(kk))]
print(mk(kk2))
plt.subplots(figsize=(28,11),dpi=600)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20,rotation=45)

plt.fill_between([str(int(i)) for i in list(range(2001,2021))],np.array(mins),np.array(maxs),color='thistle',alpha=0.6)
plt.plot([str(int(i)) for i in list(range(2001,2021))],[0]*20,color='gray',alpha=0.4,linewidth=3)
plt.plot([str(int(i)) for i in list(range(2001,2021))],kk2,color='purple',alpha=0.7,linewidth=3,label='Global land mean albedo (ALLUMs)')
plt.plot([str(int(i)) for i in list(range(2001,2021))],[mk(kk2 )[0]*i+mk(kk2 )[1] for i in range(1,21)],color='mediumorchid',alpha=1,linewidth=1.5, linestyle='--')
plt.annotate('Slope = 0.0007 per decade',[9,mk(kk2 )[0]*9+mk(kk2 )[1]+0.011/50],fontsize=20,color='mediumorchid',rotation=math.atan(mk(kk2 )[0]/0.011*10)/3.14159*180)

####################################

GLASS=torch.load('../data/GLASS_curve.pth')
kk_GLASS=[(GLASS[0][i]-GLASS[0][0])/10000 for i in range(20)]
kk_blueSA_season=GLASS[1]
x=[str(int(i)) for i in range(2001,2021)]
kk_blueSA_season_change=GLASS[1].copy()
for i in range(20):
    kk_blueSA_season_change[:,i]-=kk_blueSA_season[:,0]
top=np.max(kk_blueSA_season_change,axis=0)/10000
bottom=np.min(kk_blueSA_season_change,axis=0)/10000

print(mk(kk_GLASS))
plt.fill_between([str(int(i)) for i in list(range(2001,2021))],np.array(bottom),np.array(top),color='orange',alpha=0.2)
plt.plot([str(int(i)) for i in list(range(2001,2021))],kk_GLASS,color='orange',alpha=1,linewidth=3,label='Global land mean albedo (GLASS V50)')
plt.plot([str(int(i)) for i in list(range(2001,2021))],[mk(kk_GLASS)[0]*i+mk(kk_GLASS)[1] for i in range(1,21)],color='orange',alpha=1,linewidth=1.5, linestyle='--')
plt.annotate('Slope = 0.0009 per decade',[9,mk(kk2 )[0]*9-mk(kk2 )[1]/3],fontsize=20,color='orange',rotation=math.atan(mk(kk2 )[0]/0.011*10)/3.14159*180)
plt.grid(linestyle='-.')
plt.legend(fontsize=20)
plt.savefig('../figure/GLASS_curve_all.pdf',dpi=600)
plt.show()

(6.76267512523994e-05, 0.0008580578405365581, 'no trend', 0.31452486905097254)
(9.379594007454516e-05, 2.6840553846962545e-05, 'no trend', 0.5812547604118179)


<Figure size 12000x6600 with 0 Axes>

In [ ]:
# Export albedo change map
kk=[]
BlueSA_01=torch.load(f'/mnt/hzy_ssd2/GLASS_result/blueSAs_2001_gt70_withuncertaintyandprior.pth')
BlueSA_20=torch.load(f'/mnt/hzy_ssd2/GLASS_result/blueSAs_2020_gt70_withuncertaintyandprior.pth')
BlueSA_dif=BlueSA_20-BlueSA_01
trf=(-180,0.05,0,90,0,-0.05)
BlueSA_dif[np.isnan(BlueSA_dif)]=0
tif_save(BlueSA_dif,'/mnt/hzy_ssd2/GLASS_result/GLASS_dif.tif',trf,p='4326',novalue=None,valuetype=6)

In [ ]:
# Export albedo change trend map
def compute_slope(x_seq, y_seq):
    if np.isnan(y_seq).any():
        return 0
    else:
        model = TheilSenRegressor(max_iter=1000)
        model.fit(x_seq.reshape(-1, 1), y_seq)
        return model.coef_[0]
def compute_slope_batch(i):
    blueSA_20y_temp=blueSA_20y[:,i*20:i*20+20]
    blueSA_20y_temp=blueSA_20y_temp.reshape((20,-1)).transpose((1,0))
    slopes=[compute_slope(x[j, :], blueSA_20y_temp[j, :]) for j in range(x.shape[0])]
    slopes=np.array(slopes).reshape(20, 7200)
    torch.save(slopes,f'/mnt/hzy_ssd2/GLASS_result/trends_{i}_withuncertaintyandprior.pth')
blueSA_20y=[]
for y in tqdm(range(2001,2021)):
    blueSA_y=torch.load(f'/mnt/hzy_ssd2/GLASS_result/blueSAs_{y}_gt70_withuncertaintyandprior.pth')
    blueSA_20y.append(blueSA_y)
blueSA_20y=np.stack(blueSA_20y,axis=0)
x=(np.array(list(range(1,21))).reshape((-1,1))*np.ones((1,144000))).transpose((1,0))
i_indexs=np.array(range(180)).reshape((10,18))
i_indexs=[list(i_indexs[i]) for i in range(10)]#range(10)
i_indexs=i_indexs[-3:]
for i_index in tqdm(i_indexs):
    result = Parallel(n_jobs=18)(delayed(compute_slope_batch)(i) for i in i_index)
num=[int(os.path.basename(f).split('_')[1]) for f in glob('/mnt/hzy_ssd2/GLASS_result/trends_*_withuncertaintyandprior.pth')]
result=np.zeros((3600, 7200)).astype(np.float32)
for i in num:
    slopes=torch.load(f"/mnt/hzy_ssd2/GLASS_result/trends_{i}_withuncertaintyandprior.pth")
    result[i*20:i*20+20]=slopes
result[np.isnan(result)]=0
tif_save(result,'/mnt/hzy_ssd2/GLASS_result/GLASS_trend.tif',trf,p='4326',novalue=None,valuetype=6)